# Autodesk Inventor - ilogic
Used in Parts, Assemblies and drawings For Analyze (Read), Modify (Update) and Create/Delete content of the document.
1.  iLogic Browser - All contents of the ilogic

## Parameters

## Rules  
### Clamp Parameter to Range
Define the Limit of the particular dimension


In [ ]:
If width < 8 Then
iProperties.PartColor = "Red"
//	MessageBox.Show("Defined Width is too small; Setting to 8")
//	Logger.Warn("Defined Width is too small; Setting to 8")
width = 8
Else
iProperties.PartColor = "Particle Board"
End if

### Feature Toggle
Toggle Feature or Parts based on the Configuration

In [ ]:
// ***CompanyDecal***
Feature.IsActive("CompanyDecal") = HasDecal

### Project Properties Update
Properties of the .ipt or .iam Files

In [ ]:
If String.IsNullOrEmpty(iProperties.Value("Project", "Project")) Then
 Dim newString = InputBox("Customer project ID, or None", "Project", "")
 If (Not String.IsNullOrEmpty(newString)) Then
 iProperties.Value("Project", "Project") = newString
 End If
End If

## Forms 
Interact with (customize, configure) a model
Can Add parameters, Combo box values, Run the rule from form


## Events Triggers
Can Trigger based on the event happen in the document

In [ ]:
//Trigger based on the Props value at time
If String.IsNullOrEmpty(iProperties.Value("Project", "Project")) Then
 Dim newString = InputBox("Customer project ID, or None", "Project", "")
 If (Not String.IsNullOrEmpty(newString)) Then
 iProperties.Value("Project", "Project") = newString
 End If
End If

In [ ]:
//Update Project Description
Dim rndLength = RoundToFraction(length, 1 / 8, RoundingMethod.Round)
Dim rndWidth = RoundToFraction(width, 1 / 8, RoundingMethod.Round)
Dim rndHeight = RoundToFraction(height, 1/8, RoundingMethod.Round)
iProperties.Value("Project", "Description") = rndLength & " x " & rndWidth & " x " & rndHeight

## Assembly
Push params of Asssembly into the appropiate params of child components. 
#### Kind of Top-Down Design
It’s a one-way transfer. If you go into the IPT files, you can “manually” change the parameter values and the IAM won’t be aware of it. 
(Updates from Assembly to part is only possible)

In [ ]:
Parameter("box", "length") = length
Parameter("box", "width") = width
Parameter("box", "thickness") = thickness
Parameter("box", "height") = height - coverHeight
	
Parameter("cover", "height") = coverHeight
Parameter("cover", "length") = length
Parameter("cover", "width") = width
Parameter("cover", "thickness") = thickness

### LOD - Level Of Details
In the “Model” browser --> Expand “Representations” --> Right-click on “Level of Detail: Master”, and select “New Level of Detail”

In [ ]:
//Component Suppress based on user Parameters
Component.IsActive("cover") = hasCover

1.  When the cover is NOT present, we want the box’s height to be the total height (i.e., not subtract the coverHeight)

In [ ]:
if (hasCover) then
    Parameters("box", "Height") = Height - CoverHeight
Else
    Parameters("box", "Height") = Height
End if

1.  Create Constraint Only when the component is present
2.  when there is no-cover, there’s really no “cover” component at all! It has really been deleted. So all of the parameter-setting statements, like “Parameter(“cover”, “height”) = coverHeight”, will fail. 

In [ ]:
ThisAssembly.BeginManage()

If (hasCover) Then
	Dim coverPos = ThisAssembly.Geometry.Matrix(1, 0, 0, 12.279323679811,
                                            0, -0.367784383904, -0.929911096265, 36.516478555681,
                                            0, 0.929911096265, -0.367784383904, 3.867390889031,
                                            0, 0, 0, 1)
	Dim cover = Components.Add("cover", "cover.ipt", coverPos)

	Constraints.AddFlush("flushLeft", "cover", "leftFace", "box", "wpLeft")
	Constraints.AddMate("hinge", "cover", "hingeEdge", "box", "hingeAxis",
	                    solutionType := MateConstraintSolutionTypeEnum.kUndirectedSolutionType)

End If

If (hasCover) Then
    Parameter("box", "height") = height - coverHeight
    Parameter("cover", "height") = coverHeight
    Parameter("cover", "length") = length
    Parameter("cover", "width") = width
    Parameter("cover", "thickness") = thickness
Else
    Parameter("box", "height") = height
End If

Update Configuration list - Based On the Excel File 

In [ ]:
Dim configs As New ArrayList

Dim dbpath As String = System.IO.Path.Combine(ThisDoc.Path, "standardSizes.xlsx")
GoExcel.Open(dbpath, "Sheet1")

For i = 2 To 100  'Skip header row.  And don't go on an infinite loop, just in case there's some error.
	Dim m As String 
	m = GoExcel.CellValue("A" & i)
	If (m = "") Then
		Exit For
	End If
	configs.Add(m)
	Logger.Debug(m)
Next i
configs.Add("Custom")

MultiValue.List("ConfigurationName") = configs

### Enviroment Variables 
#### UseLibXL in Variable Name and Variable Value in 1
For Faster Loading of Excel Files
(a) It can only read cells with data in them – it doesn’t execute any formulas. 
(b) It doesn’t provide access to the Excel COM interface, so you can only access the spreadsheet using the iLogic GoExcel functions. 
(c) It applies to the whole Inventor session, so all other uses of spreadsheets will also be subject to these restrictions.
####  Default behavior when using Forge Design Automation API for Inventor and also when using Configurator 360.

In [ ]:
If (ConfigurationName = "Custom") Then
Logger.Debug("Custom settings -- no changes applied.")
IsCustom = True
Else
Dim xl_file As String = System.IO.Path.Combine(ThisDoc.Path, "standardSizes.xlsx")
Logger.Trace("Excel file is: " & xl_file)
GoExcel.FindRowStart = 2
i = GoExcel.FindRow(xl_file, "Sheet1", "Name", "=", ConfigurationName) 'case-sensitive
If (i < 0) Then
    MessageBox.Show("Standard configuration named '" & ConfigurationName & "' was not found.")
Else
    IsCustom = False
    Dim yesno As String = GoExcel.CellValue("G" & i) 'GoExcel.CurrentRowValue("Cover") 
    hasCover = (String.Compare(yesno, "Yes", True) = 0)
    Dim vals = GoExcel.CellValues("B" & i, "F" & i)
    length = vals(0)
    width = vals(1)
    height = vals(2)
    coverHeight = vals(3)
    thickness = vals(4)
End If
End If

In [ ]:
//can put this anywhere in your rule and iLogic will update the model when done
iLogicVb.UpdateWhenDone = True

## Units, especially when mixing API and iLogic. API centimeters, iLogic is doc-units